# Task 7 (Metode Word2Vec)

## Load Data

In [17]:
#install library
!pip install sastrawi
!pip install swifter
!pip install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
#import library
import pandas as pd
import numpy as np
from string import punctuation
import re
import nltk

In [22]:
#load data
df = pd.read_csv("https://raw.githubusercontent.com/triasmidwi/prosaindata/main/dataset/Data_TA.csv", sep=';')
df.head()

,NPM,Judul,Abstrak,Prodi,Label
0,40411100468,PERANCANGAN DAN IMPLEMENTASI SISTEM DATABASE T...,Sistem informasi akademik (SIAKAD) merupaka...,Teknik Informatika,RPL
1,40411100476,APLIKASI KONTROL DAN MONITORING JARINGAN KOMPU...,Berjalannya koneksi jaringan komputer dengan l...,Teknik Informatika,RPL
2,40411100480,RANCANG BANGUN APLIKASI PROXY SERVER UNTUKENKR...,Web server adalah sebuah perangkat lunak serve...,Teknik Informatika,RPL
3,70411100070,SISTEM PENDUKUNG KEPUTUSAN OPTIMASI PENJADWALA...,Penjadwalan kuliah di Perguruan Tinggi me...,Teknik Informatika,komputasi
4,80411100115,SISTEM AUGMENTED REALITY ANIMASI BENDA BERGERA...,Seiring perkembangan teknologi yang ada diduni...,Teknik Informatika,komputasi


In [23]:
df.shape

(994, 5)

In [24]:
df.isnull().sum()

NPM        141
Judul      147
Abstrak    166
Prodi      141
Label      184
dtype: int64

In [25]:
df = df.dropna()

In [26]:
df.shape

(810, 5)

In [27]:
df.isnull().sum()

NPM        0
Judul      0
Abstrak    0
Prodi      0
Label      0
dtype: int64

In [28]:
df['Label'].value_counts()

komputasi     571
RPL           198
PBA            39
komputasi       1
Komputasi       1
Name: Label, dtype: int64

## Preprocessing

### 1. Symbol & Punctuation Removal, case folding

Pada Tahap ini preprocessing yang dilakukan yaitu menghilangkan simbol dan tanda baca, serta melakukan case folding yaitu merubah seluruh huruf yang ada pada data menjadi huruf kecil

In [29]:
#proses menghilangkan simbol dan emoji
def remove_text_special (text):
  text = text.replace('\\t',"").replace('\\n',"").replace('\\u',"").replace('\\',"")
  text = text.encode('ascii', 'replace').decode('ascii')
  return text.replace("http://"," ").replace("https://", " ")
df['Abstrak'] = df['Abstrak'].apply(remove_text_special)
print(df['Abstrak'])

0      Sistem  informasi  akademik  (SIAKAD) merupaka...
1      Berjalannya koneksi jaringan komputer dengan l...
2      Web server adalah sebuah perangkat lunak serve...
3      Penjadwalan  kuliah  di  Perguruan  Tinggi  me...
4      Seiring perkembangan teknologi yang ada diduni...
                             ...                        
848    Investasi saham selama ini memiliki resiko ker...
849    Information Retrieval (IR) merupakan pengambil...
850    Klasifikasi citra merupakan proses pengelompok...
851    Identifikasi atribut pejalan kaki merupakan sa...
852    Topik deteksi objek telah menarik perhatian ya...
Name: Abstrak, Length: 810, dtype: object


In [30]:
#menghilangkan tanda baca
def remove_tanda_baca(text):
  text = re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",text)
  return text

df['Abstrak'] = df['Abstrak'].apply(remove_tanda_baca)
df['Abstrak'].head(20)

0     Sistem  informasi  akademik   SIAKAD  merupaka...
1     Berjalannya koneksi jaringan komputer dengan l...
2     Web server adalah sebuah perangkat lunak serve...
3     Penjadwalan  kuliah  di  Perguruan  Tinggi  me...
4     Seiring perkembangan teknologi yang ada diduni...
5     Gerak pekerja ada pada game yang memiliki genr...
6     Perkembangan game yang semakin pesat  memberik...
7     Sistem pengenalan wajah adalah suatu sistem un...
8     Teknologi mobile game beroperating system open...
9     Kantor Badan Kepegawaian kota Bangkalan adalah...
10    Penyusunan Sistem Informasi Dinas Perindustria...
11    Perusahaan pemerintah maupun swasta mempunyai ...
12    Pelayanan makanan bagi pasien rawat inap di Ru...
13    Penyusunan Sistem Pendukung Keputusan pemiliha...
14    Sidik jari adalah salah satu karakteristik fis...
15    Di Indonesia masalah perkembangan gizi adalah ...
16    Pengenalan tulisan tangan merupakan topik pene...
17    Citra senyum merupakan salah satu fitur bi

In [31]:
#proses menghilangkan angka
def remove_numbers (text):
  return re.sub(r"\d+", "", text)
df['Abstrak'] = df['Abstrak'].apply(remove_numbers)
df['Abstrak']

0      Sistem  informasi  akademik   SIAKAD  merupaka...
1      Berjalannya koneksi jaringan komputer dengan l...
2      Web server adalah sebuah perangkat lunak serve...
3      Penjadwalan  kuliah  di  Perguruan  Tinggi  me...
4      Seiring perkembangan teknologi yang ada diduni...
                             ...                        
848    Investasi saham selama ini memiliki resiko ker...
849    Information Retrieval  IR  merupakan pengambil...
850    Klasifikasi citra merupakan proses pengelompok...
851    Identifikasi atribut pejalan kaki merupakan sa...
852    Topik deteksi objek telah menarik perhatian ya...
Name: Abstrak, Length: 810, dtype: object

In [32]:
#proses casefolding
def casefolding(Comment):
  Comment = Comment.lower()
  return Comment
df['Abstrak'] = df['Abstrak'].apply(casefolding)
df['Abstrak']

0      sistem  informasi  akademik   siakad  merupaka...
1      berjalannya koneksi jaringan komputer dengan l...
2      web server adalah sebuah perangkat lunak serve...
3      penjadwalan  kuliah  di  perguruan  tinggi  me...
4      seiring perkembangan teknologi yang ada diduni...
                             ...                        
848    investasi saham selama ini memiliki resiko ker...
849    information retrieval  ir  merupakan pengambil...
850    klasifikasi citra merupakan proses pengelompok...
851    identifikasi atribut pejalan kaki merupakan sa...
852    topik deteksi objek telah menarik perhatian ya...
Name: Abstrak, Length: 810, dtype: object

### 2. Tokenizing
Pada tahap ini preprocessing yang dilakukan adalah tokenizing. Tokenizing adalah metode untuk melakukan pemisahan kata dalam suatu kalimat dengan tujuan untuk proses analisis teks lebih lanjut

In [33]:
#proses tokenisasi
# from nltk.tokenize import TweetTokenizer
nltk.download('punkt')
# def word_tokenize(text):
#   tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
#   return tokenizer.tokenize(text)

df['review_token'] = df['Abstrak'].apply(lambda sentence: nltk.word_tokenize(sentence))
df['review_token']

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


0      [sistem, informasi, akademik, siakad, merupaka...
1      [berjalannya, koneksi, jaringan, komputer, den...
2      [web, server, adalah, sebuah, perangkat, lunak...
3      [penjadwalan, kuliah, di, perguruan, tinggi, m...
4      [seiring, perkembangan, teknologi, yang, ada, ...
                             ...                        
848    [investasi, saham, selama, ini, memiliki, resi...
849    [information, retrieval, ir, merupakan, pengam...
850    [klasifikasi, citra, merupakan, proses, pengel...
851    [identifikasi, atribut, pejalan, kaki, merupak...
852    [topik, deteksi, objek, telah, menarik, perhat...
Name: review_token, Length: 810, dtype: object

### 3. Word Normalization
Pada tahap ini yang dilakukan yaitu normalisasi pada data. Hal tersebut dilakukan untuk merubah kata yang tidak baku menjadi kata baku

In [34]:
#Normalisasi kata tidak baku
normalize = pd.read_csv("https://raw.githubusercontent.com/triasmidwi/prosaindata/main/dataset/Normalization%20Data.csv")

normalize_word_dict = {}

for row in normalize.iterrows():
  if row[0] not in normalize_word_dict:
    normalize_word_dict[row[0]] = row[1]

def normalized_term(comment):
  return [normalize_word_dict[term] if term in normalize_word_dict else term for term in comment]

df['comment_normalize'] = df['review_token'].apply(normalized_term)
df['comment_normalize'].head(20)

0     [sistem, informasi, akademik, siakad, merupaka...
1     [berjalannya, koneksi, jaringan, komputer, den...
2     [web, server, adalah, sebuah, perangkat, lunak...
3     [penjadwalan, kuliah, di, perguruan, tinggi, m...
4     [seiring, perkembangan, teknologi, yang, ada, ...
5     [gerak, pekerja, ada, pada, game, yang, memili...
6     [perkembangan, game, yang, semakin, pesat, mem...
7     [sistem, pengenalan, wajah, adalah, suatu, sis...
8     [teknologi, mobile, game, beroperating, system...
9     [kantor, badan, kepegawaian, kota, bangkalan, ...
10    [penyusunan, sistem, informasi, dinas, perindu...
11    [perusahaan, pemerintah, maupun, swasta, mempu...
12    [pelayanan, makanan, bagi, pasien, rawat, inap...
13    [penyusunan, sistem, pendukung, keputusan, pem...
14    [sidik, jari, adalah, salah, satu, karakterist...
15    [di, indonesia, masalah, perkembangan, gizi, a...
16    [pengenalan, tulisan, tangan, merupakan, topik...
17    [citra, senyum, merupakan, salah, satu, fi

### 4. Stopwords Removal
Pada tahap ini preprocessing yang dilakukan adalah menghilangkan kata yang tidak penting. Stopwords removal dilakukan 2 kali, yang pertama berdasarkan korpus yang ada di library python yaitu nltk, yang kedua berdasarkan file 'list_stopwords'

In [35]:
#Stopword Removal
nltk.download('stopwords')
from nltk.corpus import stopwords
txt_stopwords = stopwords.words('indonesian')

def stopwords_removal(filtering) :
  filtering = [word for word in filtering if word not in txt_stopwords]
  return filtering

df['stopwords_removal'] = df['comment_normalize'].apply(stopwords_removal)
df['stopwords_removal'].head(20)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


0     [sistem, informasi, akademik, siakad, sistem, ...
1     [berjalannya, koneksi, jaringan, komputer, lan...
2     [web, server, perangkat, lunak, server, berfun...
3     [penjadwalan, kuliah, perguruan, kompleks, per...
4     [seiring, perkembangan, teknologi, didunia, mu...
5     [gerak, pekerja, game, memiliki, genre, rts, r...
6     [perkembangan, game, pesat, alternative, pemin...
7     [sistem, pengenalan, wajah, sistem, mengenali,...
8     [teknologi, mobile, game, beroperating, system...
9     [kantor, badan, kepegawaian, kota, bangkalan, ...
10    [penyusunan, sistem, informasi, dinas, perindu...
11    [perusahaan, pemerintah, swasta, tujuan, dicap...
12    [pelayanan, makanan, pasien, rawat, inap, ruma...
13    [penyusunan, sistem, pendukung, keputusan, pem...
14    [sidik, jari, salah, karakteristik, fisik, man...
15    [indonesia, perkembangan, gizi, yangperlu, per...
16    [pengenalan, tulisan, tangan, topik, penelitia...
17    [citra, senyum, salah, fitur, biometrik, d

In [36]:
#stopword removal 2
data_stopwords = pd.read_csv("https://raw.githubusercontent.com/triasmidwi/prosaindata/main/dataset/list_stopwords.csv")
print(data_stopwords)

def stopwords_removal2(filter) :
  filter = [word for word in filter if word not in data_stopwords]
  return filter

df['stopwords_removal_final'] = df['stopwords_removal'].apply(stopwords_removal2)
df['stopwords_removal_final'].head(20)

Empty DataFrame
Columns: [kah;samping;kiri;maju;mundur;orang kedua;youtube;nya;non;gaada;hehe;heheee;haha;bf;dll;un;safa;ida;idak;aja;kk;wkwkk;gimana;lihat;duel;boong;ya;coba;anjing;pilihan;sandi;seklai;anggi;tetima;ubtik;tai;t;f;a;v;lt;cq;fd;e;h;da;diba;ax;babu;tombol;kantuk;kntl;sherina;ter;eh;kitaudah;cari;daan;ketemu;sdang;luring;pas;luring;bilang;log;in;kembaliterimah;nge;lo;dakwahlbh;klan;detik;tolol;kaka;kakk;kakak;silah;mbaaak;mbak;kn;fasid;solo;loh;bagusbagus;an;nan;babi;negara;m;t;o;p;ii;nyacuma;putarpilihdengerinklo;sia;cv;katai;luring;log;tank;am;radio;simple;buk;bu;si;sih;daring;blibli;lainnya;keluarga;sim;mantaplahnice;luring;garena;garax;barca;yah;oh;heeh;rate;si;ini saya;resso;kaya;deh;pdkesmas;woi;kontol;loh;interface;gitu;mahasiswa;kobo;karnaeru;kenama;na;dulusetelah;berhasilsy;job;trm;ks;sangattttt;anak;ko;leon;hem;alhamdulillah;hai;chrome;book;thank;gandoss;lagulagunya;seharihari;laki;mah;aaaa;wkwk;iklanmau;mulumaksudnya;menh;maura;dafam;zakia;savita;full;dambil;yar

0     [sistem, informasi, akademik, siakad, sistem, ...
1     [berjalannya, koneksi, jaringan, komputer, lan...
2     [web, server, perangkat, lunak, server, berfun...
3     [penjadwalan, kuliah, perguruan, kompleks, per...
4     [seiring, perkembangan, teknologi, didunia, mu...
5     [gerak, pekerja, game, memiliki, genre, rts, r...
6     [perkembangan, game, pesat, alternative, pemin...
7     [sistem, pengenalan, wajah, sistem, mengenali,...
8     [teknologi, mobile, game, beroperating, system...
9     [kantor, badan, kepegawaian, kota, bangkalan, ...
10    [penyusunan, sistem, informasi, dinas, perindu...
11    [perusahaan, pemerintah, swasta, tujuan, dicap...
12    [pelayanan, makanan, pasien, rawat, inap, ruma...
13    [penyusunan, sistem, pendukung, keputusan, pem...
14    [sidik, jari, salah, karakteristik, fisik, man...
15    [indonesia, perkembangan, gizi, yangperlu, per...
16    [pengenalan, tulisan, tangan, topik, penelitia...
17    [citra, senyum, salah, fitur, biometrik, d

### 5. Stemming
Pada tahap ini preprocessing yang dilakukan adalah stemming. Stemming adalah proses pemetaan dan penguraian bentuk dari suatu kata menjadi bentuk kata dasarnya.

In [37]:
#proses stem
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
# from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import string
import swifter
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemming (term):
  return stemmer.stem(term)

term_dict = {}

for document in df['stopwords_removal_final']:
  for term in document:
    if term not in term_dict:
      term_dict[term] = ''

for term in term_dict.keys():
  term_dict[term] = stemming(term)

def get_stemming(document):
  return [term_dict[term] for term in document]

In [38]:
df['stemming'] = df['stopwords_removal_final'].swifter.apply(get_stemming)

Pandas Apply:   0%|          | 0/810 [00:00<?, ?it/s]

In [39]:
print(df['stemming'])

0      [sistem, informasi, akademik, siakad, sistem, ...
1      [jalan, koneksi, jaring, komputer, lancar, gan...
2      [web, server, perangkat, lunak, server, berfun...
3      [jadwal, kuliah, guru, kompleks, masalah, vari...
4      [iring, kembang, teknologi, dunia, muncul, tek...
                             ...                        
848    [investasi, saham, milik, resiko, rugi, dikare...
849    [information, retrieval, ir, ambil, informasi,...
850    [klasifikasi, citra, proses, kelompok, piksel,...
851    [identifikasi, atribut, pejal, kaki, salah, te...
852    [topik, deteksi, objek, tarik, perhati, kemban...
Name: stemming, Length: 810, dtype: object


In [40]:
df.head(20)

,NPM,Judul,Abstrak,Prodi,Label,review_token,comment_normalize,stopwords_removal,stopwords_removal_final,stemming
0,40411100468,PERANCANGAN DAN IMPLEMENTASI SISTEM DATABASE T...,sistem informasi akademik siakad merupaka...,Teknik Informatika,RPL,"[sistem, informasi, akademik, siakad, merupaka...","[sistem, informasi, akademik, siakad, merupaka...","[sistem, informasi, akademik, siakad, sistem, ...","[sistem, informasi, akademik, siakad, sistem, ...","[sistem, informasi, akademik, siakad, sistem, ..."
1,40411100476,APLIKASI KONTROL DAN MONITORING JARINGAN KOMPU...,berjalannya koneksi jaringan komputer dengan l...,Teknik Informatika,RPL,"[berjalannya, koneksi, jaringan, komputer, den...","[berjalannya, koneksi, jaringan, komputer, den...","[berjalannya, koneksi, jaringan, komputer, lan...","[berjalannya, koneksi, jaringan, komputer, lan...","[jalan, koneksi, jaring, komputer, lancar, gan..."
2,40411100480,RANCANG BANGUN APLIKASI PROXY SERVER UNTUKENKR...,web server adalah sebuah perangkat lunak serve...,Teknik Informatika,RPL,"[web, server, adalah, sebuah, perangkat, lunak...","[web, server, adalah, sebuah, perangkat, lunak...","[web, server, perangkat, lunak, server, berfun...","[web, server, perangkat, lunak, server, berfun...","[web, server, perangkat, lunak, server, berfun..."
3,70411100070,SISTEM PENDUKUNG KEPUTUSAN OPTIMASI PENJADWALA...,penjadwalan kuliah di perguruan tinggi me...,Teknik Informatika,komputasi,"[penjadwalan, kuliah, di, perguruan, tinggi, m...","[penjadwalan, kuliah, di, perguruan, tinggi, m...","[penjadwalan, kuliah, perguruan, kompleks, per...","[penjadwalan, kuliah, perguruan, kompleks, per...","[jadwal, kuliah, guru, kompleks, masalah, vari..."
4,80411100115,SISTEM AUGMENTED REALITY ANIMASI BENDA BERGERA...,seiring perkembangan teknologi yang ada diduni...,Teknik Informatika,komputasi,"[seiring, perkembangan, teknologi, yang, ada, ...","[seiring, perkembangan, teknologi, yang, ada, ...","[seiring, perkembangan, teknologi, didunia, mu...","[seiring, perkembangan, teknologi, didunia, mu...","[iring, kembang, teknologi, dunia, muncul, tek..."
5,70411100007,Gerak Pekerja Pada Game Real Time Strategy Men...,gerak pekerja ada pada game yang memiliki genr...,Teknik Informatika,komputasi,"[gerak, pekerja, ada, pada, game, yang, memili...","[gerak, pekerja, ada, pada, game, yang, memili...","[gerak, pekerja, game, memiliki, genre, rts, r...","[gerak, pekerja, game, memiliki, genre, rts, r...","[gerak, kerja, game, milik, genre, rts, real, ..."
6,70411100126,RANCANG BANGUN GAME PERAWATAN SAPI KARAPAN MEN...,perkembangan game yang semakin pesat memberik...,Teknik Informatika,komputasi,"[perkembangan, game, yang, semakin, pesat, mem...","[perkembangan, game, yang, semakin, pesat, mem...","[perkembangan, game, pesat, alternative, pemin...","[perkembangan, game, pesat, alternative, pemin...","[kembang, game, pesat, alternative, minat, ben..."
7,70411100109,EKSTRAKSI FITUR BERBASIS TWO DIMENSIONAL LINEA...,sistem pengenalan wajah adalah suatu sistem un...,Teknik Informatika,komputasi,"[sistem, pengenalan, wajah, adalah, suatu, sis...","[sistem, pengenalan, wajah, adalah, suatu, sis...","[sistem, pengenalan, wajah, sistem, mengenali,...","[sistem, pengenalan, wajah, sistem, mengenali,...","[sistem, kenal, wajah, sistem, nali, identitas..."
8,80411100083,IMPLEMENTASI ALGORITMA PRIM DAN DEPTH FIRST ...,teknologi mobile game beroperating system open...,Teknik Informatika,komputasi,"[teknologi, mobile, game, beroperating, system...","[teknologi, mobile, game, beroperating, system...","[teknologi, mobile, game, beroperating, system...","[teknologi, mobile, game, beroperating, system...","[teknologi, mobile, game, beroperating, system..."
9,70411100092,Perancangan Sistem Informasi Badan Kepegawaian...,kantor badan kepegawaian kota bangkalan adalah...,Teknik Informatika,RPL,"[kantor, badan, kepegawaian, kota, bangkalan, ...","[kantor, badan, kepegawaian, kota, bangkalan, ...","[kantor, badan, kepegawaian, kota, bangkalan, ...

## Word Embedding Word2vec CBOW

In [41]:
import gensim
from gensim.models import Word2Vec

In [42]:
# Create CBOW model
model1 = gensim.models.Word2Vec(df['stemming'], min_count = 1,
                              vector_size = 100, window = 5)
 
# Print results
print("Cosine similarity between 'teknologi' " +
               "and 'system' - CBOW : ",
    model1.wv.similarity('teknologi', 'system'))
     
print("Cosine similarity between 'teknologi' " +
                 "and 'mobile' - CBOW : ",
      model1.wv.similarity('teknologi', 'mobile'))

Cosine similarity between 'teknologi' and 'system' - CBOW :  0.9665671
Cosine similarity between 'teknologi' and 'mobile' - CBOW :  0.9920404


In [43]:
#getting similar word
sim_words = model1.wv.most_similar('teknologi')
sim_words

[('pesat', 0.9981826543807983),
 ('kembang', 0.9962599277496338),
 ('media', 0.9938992261886597),
 ('edukasi', 0.993705153465271),
 ('reality', 0.9927350878715515),
 ('game', 0.9923123717308044),
 ('mobile', 0.9920405149459839),
 ('interaktif', 0.9918705821037292),
 ('augmented', 0.9915143251419067),
 ('didik', 0.9906712770462036)]